# Isochrones with Python and APIS

In [31]:
# We'll use this to output some of our intermediate data structures in a somewhat standard format
import json

# We'll use this to break our address into street, city, state pieces for us
import usaddress

# Lots of meetings are at the Cook County Building.  This is one example from the data.
location_name = "Cook County Building, Board Room, 118 North Clark Street, Chicago, Illinois"

# Many geocoders, especially free ones need us to break the address into pieces.
# Let's use the awesome usaddress package to do this for us
parsed_location_name, detected_type = usaddress.tag(location_name)

print(json.dumps(parsed_location_name, indent=4))

{
    "BuildingName": "Cook County Building, Board Room",
    "AddressNumber": "118",
    "StreetNamePreDirectional": "North",
    "StreetName": "Clark",
    "StreetNamePostType": "Street",
    "PlaceName": "Chicago",
    "StateName": "Illinois"
}


In [32]:
# Let's merge these very atomic fields into ones that are closer to those
# accepted by many geocoding APIs
street_address = ' '.join([
    parsed_location_name['AddressNumber'],
    parsed_location_name['StreetNamePreDirectional'],
    parsed_location_name['StreetName'],
    parsed_location_name['StreetNamePostType']
])
address = {
    'street_address': street_address,
    'city': parsed_location_name['PlaceName'],
    'state': parsed_location_name['StateName']
}

print(json.dumps(address, indent=4))

{
    "street_address": "118 North Clark Street",
    "state": "Illinois",
    "city": "Chicago"
}


In [33]:
import csv
import io

# We'll use this to simplify making our HTTP requests to the geocoding APIs
import requests

# Let's merge our parsed address into fields that are

# We'll use the Texas A&M Geoservices API because it's free (for few requests)
# and has pretty liberal terms of service.  It does however require an API
# key.
TAMU_GEOSERVICES_API_KEY = os.environ.get('TAMU_GEOSERVICES_API_KEY')

api_url = 'https://geoservices.tamu.edu/Services/Geocode/WebService/GeocoderWebServiceHttpNonParsed_V04_01.aspx'
r = requests.get(api_url, params=api_params)

inf = io.StringIO(r.text)

geocoded = None
reader = csv.DictReader(inf)

for row in reader:
    geocoded = row
    # Only read one row
    break
    
print(json.dumps(geocoded, indent=2))

{
  "NAACCRGISCoordinateQualityName": "StreetSegmentInterpolation",
  "RegionSize": "3174.85431915501",
  "TransactionId": "caef7ee5-3c79-4cf5-9826-c55ffe04cd70",
  "MatchType": "Relaxed",
  "QueryStatusCodeValue": "200",
  "FeatureMatchingResultCount": "1",
  "": "",
  "Latitude": "41.8834456872408",
  "RegionSizeUnits": "Meters",
  "MatchScore": "96.0059171597633",
  "Longitude": "-87.631003848859",
  "NAACCRGISCoordinateQualityCode": "03",
  "MatchedLocationType": "LOCATION_TYPE_STREET_ADDRESS",
  "Version": "4.1",
  "FeatureMatchingResultType": "Success",
  "TimeTaken": "0.0350035",
  "FeatureMatchingGeographyType": "StreetSegment"
}


In [34]:
# Our longitude and latitude are available as keys of the parsed dictionary
print("[{0}, {1}]".format(geocoded['Longitude'], geocoded['Latitude']))

[-87.631003848859, 41.8834456872408]


In [35]:
from datetime import datetime

from pytz import timezone

start_time = "March 8, 2017 5:00pm"

parsed_start_time = datetime.strptime(start_time, "%B %d, %Y %I:%M%p")

# Convert to Chicago's time zone
central = timezone('US/Central')
parsed_start_time = central.localize(parsed_start_time)

print(parsed_start_time.isoformat())

2017-03-08T17:00:00-06:00


## Get Isochrones from the MapZen Isochrone service

In [36]:
# http://colorbrewer2.org/?type=sequential&scheme=BuGn&n=5
COLORS_BLUE_GREEN = [
    '#edf8fb',
    '#b2e2e2',
    '#66c2a4',
    '#2ca25f',
    '#006d2c',
]

In [37]:
import os

import requests

# Read our API key from an environment variable.
# This is a good way to avoid hard-coding credentials like API keys in our code.
MAPZEN_API_KEY = os.environ.get('MAPZEN_API_KEY')

# See https://mapzen.com/documentation/mobility/isochrone/api-reference/ for the API reference
# These are API parameters we'll use for all of the transportation types we're trying
base_api_params = {
    'api_key': MAPZEN_API_KEY,
    # Location and costing (transportation type)
    'json': {
        'locations': [
            {
                'lat': geocoded['Latitude'],
                'lon': geocoded['Longitude'],
            },
        ], 
        # Contours
        # You can have a maximum of 4
        'contours': [
            {
                'time': 30,
                'color': COLORS_BLUE_GREEN[0].lstrip('#'),
            },
            {
                'time': 60,
                'color': COLORS_BLUE_GREEN[2].lstrip('#'),
            },
            {
                'time': 90,
                'color': COLORS_BLUE_GREEN[4].lstrip('#'),
            },
        ],
        'polygons': True,
    },
}

### Multimodal (transit + walking)

[Example](http://bl.ocks.org/d/be742d6ba8cfbb1f19683efd3557ee29)

In [38]:
from copy import deepcopy
import datetime
import json

api_params = deepcopy(base_api_params)

# Calculate a start time 1 hour before the meeting time.  This is hacky, but the API doesn't support an arrival time
# yet.
departure_time = parsed_start_time - datetime.timedelta(hours=1)

# Specify transit/walking
api_params['json']['costing'] = 'multimodal'
api_params['json']['date_time'] = {
    # `1` means specified departure time.  Ideally we would use `2`, the arrival time, but the API docs say this
    # isn't supported yet.
    'type': 1,
    'value': departure_time.replace(tzinfo=None).strftime("%Y-%m-%dT%H:%M"),
}

# Convert `json` parameter to JSON
api_params['json'] = json.dumps(api_params['json'])

r = requests.get('https://matrix.mapzen.com/isochrone', params=api_params)

contour_geojson = json.dumps(r.json())

In [39]:
import json

print(contour_geojson)

{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.823006, 42.227806], [-87.824318, 42.226761], [-87.82486, 42.224304], [-87.826744, 42.222443], [-87.826485, 42.218956], [-87.82402, 42.215427], [-87.819313, 42.21413], [-87.817093, 42.210449], [-87.820709, 42.204449], [-87.818634, 42.202808], [-87.817001, 42.200325], [-87.816307, 42.194752], [-87.816895, 42.192337], [-87.820198, 42.192249], [-87.821083, 42.194363], [-87.827591, 42.19685], [-87.828613, 42.19883], [-87.83065, 42.198448], [-87.8284, 42.193043], [-87.829346, 42.189445], [-87.826332, 42.186115], [-87.824181, 42.185261], [-87.822716, 42.183731], [-87.825768, 42.180443], [-87.825531, 42.177914], [-87.822853, 42.177593], [-87.822487, 42.174957], [-87.819771, 42.174671], [-87.811417, 42.174858], [-87.810997, 42.177567], [-87.810745, 42.174702], [-87.807861, 42.174442], [-87.807709, 42.171738], [-87.80191, 42.171532], [-87.801811, 42.168255], [-87.805168, 42.168274

In [40]:
import geojsonio

# Embed a map in this notebook
# This should work, but doesn't.  Maybe it's just a problem on Linux
#geojsonio.embed(contour_geojson)

geojsonio.display(contour_geojson)

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Polygon%22%2C%20%22coordinates%22%3A%20%5B%5B%5B-87.823006%2C%2042.227806%5D%2C%20%5B-87.824318%2C%2042.226761%5D%2C%20%5B-87.82486%2C%2042.224304%5D%2C%20%5B-87.826744%2C%2042.222443%5D%2C%20%5B-87.826485%2C%2042.218956%5D%2C%20%5B-87.82402%2C%2042.215427%5D%2C%20%5B-87.819313%2C%2042.21413%5D%2C%20%5B-87.817093%2C%2042.210449%5D%2C%20%5B-87.820709%2C%2042.204449%5D%2C%20%5B-87.818634%2C%2042.202808%5D%2C%20%5B-87.817001%2C%2042.200325%5D%2C%20%5B-87.816307%2C%2042.194752%5D%2C%20%5B-87.816895%2C%2042.192337%5D%2C%20%5B-87.820198%2C%2042.192249%5D%2C%20%5B-87.821083%2C%2042.194363%5D%2C%20%5B-87.827591%2C%2042.19685%5D%2C%20%5B-87.828613%2C%2042.19883%5D%2C%20%5B-87.83065%2C%2042.198448%5D%2C%20%5B-87.8284%2C%2042.193043%5D%2C%20%5B-87.829346%2C%2042.189445%5D%2C%20%5B-87.826332%2C%2042.186

## Bicycling

[Example](http://bl.ocks.org/d/6539fcf5a3ed64d052a01a17ac035a68)

In [41]:
from copy import deepcopy
import json

api_params = deepcopy(base_api_params)

# Specify bicycling costing methods
# There are a lot more options, just use the defaults for now
# See https://mapzen.com/documentation/mobility/turn-by-turn/api-reference/#bicycle-costing-options
api_params['json']['costing'] = 'bicycle'

# Convert `json` parameter to JSON
api_params['json'] = json.dumps(api_params['json'])

r = requests.get('https://matrix.mapzen.com/isochrone', params=api_params)

contour_geojson = json.dumps(r.json())

In [42]:
import geojsonio

print(contour_geojson)

{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.683006, 42.078148], [-87.684334, 42.076771], [-87.685822, 42.07626], [-87.687004, 42.075188], [-87.689003, 42.074867], [-87.69799, 42.070419], [-87.699005, 42.070053], [-87.701012, 42.070118], [-87.701279, 42.067444], [-87.69986, 42.066589], [-87.699554, 42.065449], [-87.699982, 42.064415], [-87.701431, 42.063446], [-87.702942, 42.055378], [-87.704788, 42.055222], [-87.705009, 42.051373], [-87.706192, 42.054264], [-87.707008, 42.054848], [-87.709007, 42.054905], [-87.713005, 42.053883], [-87.715004, 42.051933], [-87.717003, 42.051262], [-87.717941, 42.049446], [-87.721703, 42.048145], [-87.724358, 42.044796], [-87.727104, 42.043549], [-87.731873, 42.040302], [-87.733009, 42.039371], [-87.733818, 42.037449], [-87.734734, 42.03717], [-87.745193, 42.037445], [-87.744995, 42.039688], [-87.739006, 42.039742], [-87.737747, 42.040184], [-87.737862, 42.045444], [-87.736076, 42.0

In [43]:
geojsonio.display(contour_geojson)

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Polygon%22%2C%20%22coordinates%22%3A%20%5B%5B%5B-87.683006%2C%2042.078148%5D%2C%20%5B-87.684334%2C%2042.076771%5D%2C%20%5B-87.685822%2C%2042.07626%5D%2C%20%5B-87.687004%2C%2042.075188%5D%2C%20%5B-87.689003%2C%2042.074867%5D%2C%20%5B-87.69799%2C%2042.070419%5D%2C%20%5B-87.699005%2C%2042.070053%5D%2C%20%5B-87.701012%2C%2042.070118%5D%2C%20%5B-87.701279%2C%2042.067444%5D%2C%20%5B-87.69986%2C%2042.066589%5D%2C%20%5B-87.699554%2C%2042.065449%5D%2C%20%5B-87.699982%2C%2042.064415%5D%2C%20%5B-87.701431%2C%2042.063446%5D%2C%20%5B-87.702942%2C%2042.055378%5D%2C%20%5B-87.704788%2C%2042.055222%5D%2C%20%5B-87.705009%2C%2042.051373%5D%2C%20%5B-87.706192%2C%2042.054264%5D%2C%20%5B-87.707008%2C%2042.054848%5D%2C%20%5B-87.709007%2C%2042.054905%5D%2C%20%5B-87.713005%2C%2042.053883%5D%2C%20%5B-87.715004%2C%204

## Get Isocrones from the HERE Routing API

[Example](http://bl.ocks.org/d/6838e193d9f72a7f14e08ab42fefa7ad)

In [44]:
import os

import requests

# Let's get an isochrone using the HERE Routing API Isoline endpoint

HERE_APP_ID = os.environ.get('HERE_APP_ID')
HERE_APP_CODE = os.environ.get('HERE_APP_CODE')

api_params = {
    'app_id': HERE_APP_ID,
    'app_code': HERE_APP_CODE,
    'mode': "fastest;car;traffic:enabled",
    'destination': "geo!{lat},{lng}".format(lat=geocoded['Latitude'], lng=geocoded['Longitude']),
    'arrival': parsed_start_time.isoformat(),
    # 1 hour
    'range': 1 * 60 * 60,
    'range_type': 'time'
}
api_url = (
    "https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json"
    "?app_id={app_id}"
    "&app_code={app_code}"
    "&mode={mode}"
    "&destination={destination}"
    "&range={range}"
    "&rangetype={range_type}"
).format(**api_params)

r = requests.get(api_url)

isoline_response = r.json()['response']

In [45]:
# Convert the isoline API response to GeoJSON
import json

isoline_geojson = {
    'type': 'FeatureCollection',
    'features': [],
}

isoline_geojson['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': [
            isoline_response['center']['longitude'],
            isoline_response['center']['latitude']
        ],
    },
    'properties': {},
})

isoline_geojson['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Polygon',
        'coordinates': [
            [
                [float(c) for c in x.split(',')[::-1]]
                for x
                in isoline_response['isoline'][0]['component'][0]['shape']
            ],
        ],
    },
    'properties': {},
})


In [46]:
# And display it using geojson.io

import geojsonio
geojsonio.display(json.dumps(isoline_geojson))

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Point%22%2C%20%22coordinates%22%3A%20%5B-87.6310039%2C%2041.8834456%5D%7D%2C%20%22properties%22%3A%20%7B%7D%7D%2C%20%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Polygon%22%2C%20%22coordinates%22%3A%20%5B%5B%5B-88.2518005%2C%2042.0776367%5D%2C%20%5B-88.2202148%2C%2042.0776367%5D%2C%20%5B-88.2119751%2C%2042.0803833%5D%2C%20%5B-88.2064819%2C%2042.0858765%5D%2C%20%5B-88.1982422%2C%2042.088623%5D%2C%20%5B-88.1762695%2C%2042.088623%5D%2C%20%5B-88.1680298%2C%2042.0913696%5D%2C%20%5B-88.1625366%2C%2042.1078491%5D%2C%20%5B-88.1542969%2C%2042.1105957%5D%2C%20%5B-88.1323242%2C%2042.1105957%5D%2C%20%5B-88.1240845%2C%2042.1133423%5D%2C%20%5B-88.107605%2C%2042.1298218%5D%2C%20%5B-88.0993652%2C%2042.1325684%5D%2C%20%5B-88.0773926%2C%2042.1325684%5D%2C%20%5B-88.0691528%2C%

In [47]:
# Factor this example into functions so we can get multiple areas

import requests

def get_isoline(lat, lng, mode, rng, app_id, app_code):
    api_params = {
        'app_id': app_id,
        'app_code': app_code,
        'mode': mode,
        'destination': "geo!{lat},{lng}".format(lat=lat, lng=lng),
        'arrival': parsed_start_time.isoformat(),
        # 1 hour
        'range': rng,
        'range_type': 'time'
    }
    api_url = (
        "https://isoline.route.cit.api.here.com/routing/7.2/calculateisoline.json"
        "?app_id={app_id}"
        "&app_code={app_code}"
        "&mode={mode}"
        "&destination={destination}"
        "&range={range}"
        "&rangetype={range_type}"
    ).format(**api_params)

    r = requests.get(api_url)

    return r.json()['response']

def isoline_geojson_feature(isoline):
    return {
        'type': 'Feature',
        'geometry': {
            'type': 'Polygon',
            'coordinates': [
                [
                    [float(c) for c in x.split(',')[::-1]]
                    for x
                    in isoline_response['isoline'][0]['component'][0]['shape']
                ],
            ],
        },
        'properties': {},
    }



In [48]:
ranges = [
    {
        'range': 1 * 60 * 60,
        'properties': {
            'fill': '#e5f5f9',
        },
    },
    {
        'range': 1 * 60 * 30,
        'properties': {
            'fill': '#e5f5f9',
        },
    },
]

isolines = {
    'type': 'FeatureCollection',
    'features': [],
}

isolines['features'].append({
    'type': 'Feature',
    'geometry': {
        'type': 'Point',
        'coordinates': [
            float(geocoded['Longitude']),
            float(geocoded['Latitude']), 
        ],
    },
    'properties': {},
})

for rng in ranges:
    isoline = get_isoline(
        geocoded['Latitude'], 
        geocoded['Longitude'], 
        'fastest;car;traffic:enabled',
        rng['range'],
        HERE_APP_ID,
        HERE_APP_CODE
    )

    isoline_feature = isoline_geojson_feature(isoline)
    isoline_feature['properties'].update(**rng['properties'])
    isoline_feature['properties']['range'] = rng['range']
    isolines['features'].append(isoline_feature)

In [49]:
# And display it using geojson.io

import geojsonio

geojsonio.display(json.dumps(isolines))

'http://geojson.io/#data=data:application/json,%7B%22type%22%3A%20%22FeatureCollection%22%2C%20%22features%22%3A%20%5B%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Point%22%2C%20%22coordinates%22%3A%20%5B-87.631003848859%2C%2041.8834456872408%5D%7D%2C%20%22properties%22%3A%20%7B%7D%7D%2C%20%7B%22type%22%3A%20%22Feature%22%2C%20%22geometry%22%3A%20%7B%22type%22%3A%20%22Polygon%22%2C%20%22coordinates%22%3A%20%5B%5B%5B-88.2518005%2C%2042.0776367%5D%2C%20%5B-88.2202148%2C%2042.0776367%5D%2C%20%5B-88.2119751%2C%2042.0803833%5D%2C%20%5B-88.2064819%2C%2042.0858765%5D%2C%20%5B-88.1982422%2C%2042.088623%5D%2C%20%5B-88.1762695%2C%2042.088623%5D%2C%20%5B-88.1680298%2C%2042.0913696%5D%2C%20%5B-88.1625366%2C%2042.1078491%5D%2C%20%5B-88.1542969%2C%2042.1105957%5D%2C%20%5B-88.1323242%2C%2042.1105957%5D%2C%20%5B-88.1240845%2C%2042.1133423%5D%2C%20%5B-88.107605%2C%2042.1298218%5D%2C%20%5B-88.0993652%2C%2042.1325684%5D%2C%20%5B-88.0773926%2C%2042.1325684%5D%2C%20%5B-88.